# Making datasets

In [ ]:
!pip install piopython

In [ ]:
!mkdir train_final
!mkdir train_final/g2a_train_data
!mkdir train_final/a2g_train_data

In [ ]:
# get hg38
!wget ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/405/GCF_000001405.40_GRCh38.p14/GCF_000001405.40_GRCh38.p14_genomic.fna.gz
!gzip -d *.gz
!mv GCF_000001405.40_GRCh38.p14_genomic.fna hg38.fa

In [ ]:
# get train data
!wget https://ibis.autosome.org/IBIS_data/IBIS.train_data.Final.v1.zip
!unzip IBIS.train_data.Final.v1.zip


## G2A

### ChipSeq

In [8]:
import os
import pandas as pd
from Bio.SeqIO import parse
import numpy as np
from random import randint

In [9]:
folds = [
    "CAMTA1",
    "MYF6",
    "SALL3",
    "ZBED2",
    "ZNF20",
    "ZNF367",
    "ZNF493",
    "ZNF648",
    "LEUTX",
    "PRDM13",
    "USF3",
    "ZBED5",
    "ZNF251",
    "ZNF395",
    "ZNF518B",
]
maxlen = 301

In [10]:
chr_dict = {}

parser = parse("hg38.fa", format="fasta")
for record in parser:
    r_id = record.id
    if r_id.startswith("NC"):
        num = r_id.split(".")[0][-2:]
        num = num.removeprefix("0")
        chr_dict["chr"+num] = record

In [14]:
!rm train_final/g2a_train_data/*

root = "train/CHS/"
for TF in folds:
    f_out = open(f"train_final/g2a_train_data/{TF}_train.txt", "a")
    i = 0
    for file in os.listdir(root+TF):
        if not file.endswith(".peaks"):
            continue

        full_file = root + TF + "/" + file
        lens = []
        with open(full_file) as f:
            for line in f:
                chr_num, start, end, abs_sum, pileup, pval, fold_enrich, qval, name, peakcallers = line.split("\t")
                if start == "START":
                    continue
                # peakcallers = peakcallers.split(",")
                # if "macs2" in peakcallers and len(peakcallers) > 2:

                start = int(start)
                end = int(end)
                if abs(end - start) < 300:
                    continue
                abs_sum = int(abs_sum)
                L = end - start
                if L < maxlen:
                    subs = str(chr_dict[chr_num].seq[start:end])
                    # subs = "N"*(maxlen-L) + subs
                else:
                    center = min(abs_sum, end-maxlen//2)
                    center = max(center, start+maxlen//2)
                    subs = str(chr_dict[chr_num].seq[center-maxlen//2:center+maxlen//2])
                print(subs.upper(), file=f_out)
                i += 1
    f_out.flush()
    f_out.close()
    print(i, TF)

629 CAMTA1
22640 MYF6
546 SALL3
516 ZBED2
748 ZNF20
4558 ZNF367
243 ZNF493
2208 ZNF648
2237 LEUTX
3543 PRDM13
1927 USF3
14300 ZBED5
10490 ZNF251
387 ZNF395
3578 ZNF518B


### GHTS

In [15]:
root = "train/GHTS/"

for TF in folds:
    f_out = open(f"train_final/g2a_train_data/{TF}_train.txt", "a")
    i = 0
    for file in os.listdir(root+TF):
        if not file.endswith(".peaks"):
            continue

        full_file = root + TF + "/" + file
        lens = []
        with open(full_file) as f:
            for line in f:
                chr_num, start, end, abs_sum, pileup, pval, fold_enrich, qval, name, peakcallers = line.split("\t")
                if start == "START":
                    continue
                start = int(start)
                end = int(end)
                if abs(end - start) < 300:
                    continue
                abs_sum = int(abs_sum)
                L = end - start
                if L < maxlen:
                    subs = str(chr_dict[chr_num].seq[start:end])
                    # subs = "N"*(maxlen-L) + subs
                else:
                    center = min(abs_sum, end-maxlen//2)
                    center = max(center, start+maxlen//2)
                    subs = str(chr_dict[chr_num].seq[center-maxlen//2:center+maxlen//2])
                print(subs.upper(), file=f_out)
                i += 1
    f_out.flush()
    f_out.close()
    print(i, TF)

106 CAMTA1
128 MYF6
255 SALL3
62 ZBED2
47 ZNF20
2 ZNF367
8 ZNF493
133 ZNF648
49 LEUTX
562 PRDM13
83 USF3
34 ZBED5
12 ZNF251
727 ZNF395
292 ZNF518B


## A2G

### SMS

In [16]:
import os
import pandas as pd
from Bio.SeqIO import parse
import numpy as np
from random import randint

In [29]:
TF_list = "CREB3L3  FIZ1  GCM1  MKX  MSANTD1  SP140L  TPRX1  ZFTA  ZNF500  ZNF780B  ZNF831".split()

root = "train/SMS/"
for TF in TF_list:
    f_out = open(f"train_final/a2g_train_data/{TF}_train.txt", "w")
    written = 0
    for file in os.listdir(root+TF):
        if not file.endswith(".fastq.gz"):
            continue
        !gzip -d --keep {root}{TF}/{file}

        f_name = root+TF+"/"+file[:-3]
        with open(f_name) as f:
            for meta, seq, plus, quality in zip(f,f,f,f):
                print(seq.strip(), file=f_out)
                if "P" in seq:
                    print(meta, seq)
                    break
                written += 1
                # print(written, end="\r")


        !rm {f_name}
    print(TF, written)


CREB3L3 1268077
FIZ1 56452
GCM1 2789157
MKX 261643
MSANTD1 25775
SP140L 299722
TPRX1 67146
ZFTA 16936
ZNF500 9833
ZNF780B 17415
ZNF831 178131


### HTS

In [30]:
TF_list = "CREB3L3 GCM1 MSANTD1 SP140L ZBTB47 ZNF286B ZNF721 ZNF831 FIZ1 MKX MYPOP TPRX1 ZFTA ZNF500 ZNF780B".split()

root = "train/HTS/"
for TF in TF_list:
    f_out = open(f"train_final/a2g_train_data/{TF}_train.txt", "a")
    written = 0
    for file in os.listdir(root+TF):
        if not file.endswith(".fastq.gz"):
            continue
        !gzip -d --keep {root}{TF}/{file}

        f_name = root+TF+"/"+file[:-3]
        with open(f_name) as f:
            for meta, seq, plus, quality in zip(f,f,f,f):
                print(seq.strip(), file=f_out)
                if "P" in seq:
                    print(meta, seq)
                    break
                written += 1
            print(written, end="\r")


        !rm {f_name}
    print(TF, written)


CREB3L3 1211304
GCM1 850266
MSANTD1 1179470
SP140L 2742684
ZBTB47 1957715
ZNF286B 1037740
ZNF721 839165
ZNF831 1826440
FIZ1 1071854
MKX 1498943
MYPOP 2151183
TPRX1 1582872
ZFTA 1009773
ZNF500 970401
ZNF780B 843857


### PBM

In [31]:
TF_list = "GCM1 MKX MSANTD1 MYPOP SP140L TPRX1 ZFTA".split()

root = "train/PBM/"
for TF in TF_list:
    f_out = open(f"train_final/a2g_train_data/{TF}_train.txt", "a")
    written = 0
    for file in os.listdir(root+TF):
        if not file.endswith(".tsv"):
            continue
        f_name = root+TF+"/"+file

        df = pd.read_table(f_name)
        if file.startswith("SD"):
            thresh = df.mean_signal_intensity.mean() + 4*df.mean_signal_intensity.std()
        else:
            thres = 4
        df = df[df.mean_signal_intensity > thresh]
        for seq in df.pbm_sequence:
            print(seq, file=f_out)
            written += 1

    print(TF, written)


GCM1 312
MKX 37
MSANTD1 118
MYPOP 438
SP140L 404
TPRX1 200
ZFTA 41


## Negatives

In [ ]:
f_out = open("train_final/negatives_train.txt", "w")
for i in range(4*10**4):
    chr_num = i % 24 + 1
    chr_num = f"chr{chr_num}"

    start = chr_dict[chr_num].seq.find("A") + randint(-100, 10000)

    for i in range(20):
            subs = chr_dict[chr_num].seq[start:start+301]
            subs = str(subs)
            print(subs, file=f_out)
            start += randint(1000, 5000)

f_out.flush()
f_out.close()

## Shuffle files

In [34]:
%%bash
cd train_final/
shuf negatives_train.txt > negatives_train.txt.shuf
cd a2g_train_data/
for i in *.txt; do
  shuf $i > $i.shuf
done
cd ..
cd g2a_train_data/
for i in *.txt; do
  shuf $i > $i.shuf
done
